# Importa Modulos necessários
Nesse exemplo vamos importar o modulo do Pandas que vai ser necessário para abrir o dataframe no formato CSV

In [49]:
import pandas as pd
import xlrd, os, glob, re, csv, shutil

# Seta Variaveis e cria Lista
Cria as variáveis com as pastas origem e destino da conversão de XLS para CSV e cria uma lista para conversão

In [50]:
## Variaveis
PastaXLS = 'dados/DadosSalicNetXLS/'
PastaCSV = 'dados/DadosSalicNetCSV/'
lista_arquivos = os.listdir(PastaXLS)

# Lendo o conteúdo e convertendo
Vamos um for com a lista dos arquivos da pasta de arquivos XLS, com base nessa lista, vamos fazer a leitura de cada arquivo de `i` arquivo com variável `wd` que serve pro arquivo xls ser lido pelo modulo importado `xlrd` (por causa do encoding e tal não conseguiu ler direto com o `read_excel`), posteriormente lemos o `wd` com o `read_excel`. Como teremos que mudar a extenção do arquivo de `.xls` para `.csv`, criei uma váriavel `nome` para remover a extensão de `i`, posteriormente usando o `to_excel`, uso essa variavel `nome`, concatenando com `.csv` onde gero o arquivo final no formato que queremos.

Foi realizada também uma alteração dos Headers sem espaço e sem acentos, facilitando o acesso as informações, são criados arquivos temporários com o final `nh.csv` a partir dos gerados inicialmente e posteriormente eles sobrescrevem os criados inicialmente. Isso ajudou na concatenação deles para geração de um único dataframe.

In [51]:

for i in lista_arquivos:
    ano = (re.findall("dados_pj_(\d+).xls", i))[0]
    wb = xlrd.open_workbook(PastaXLS + '/' + i, encoding_override='ISO-8859-1')
    arquivoExcel = pd.read_excel(wb).assign(ano_corrente=ano)
    nome = i.rpartition('.')[0]
    arquivo = PastaCSV + '/' + nome + '.csv'
    arquivoExcel.to_csv (arquivo, index = None, header=True)

    ## Mudando o nome do header dos arquivos
    headerCagado = arquivo
    headerBacana = os.path.splitext(headerCagado)[0] + "_nh.csv"

    with open(headerCagado, newline='') as entrada, open(headerBacana, 'w', newline='') as saida:
        r = csv.reader(entrada)
        w = csv.writer(saida)

        next(r, None)
        w.writerow(['cpf_cnpj','incentivador','nro_projeto','nome_projeto','uf_projeto','valor_incentivo','ano'])

        for row in r:
            w.writerow(row)

    ## Sobrescrevendo o CSV zuado com o corrigido:
    shutil.move(headerBacana, arquivo)

# Gerando o Dataframe consolidado:
Vamos consolidar todos os arquivos `CSV` em um único dataframe, a idéia é podermos tratar os dados antes da geração do `CSV` final.

In [52]:
## Lista de todos os CSVs no diretório
lista_csvs = glob.glob(os.path.join(PastaCSV, "*.csv"))

## Criando um dataframe zerado com os Datatypes definidos
dados = pd.DataFrame()

## Inserindo todos os CSVs no dataframe criado:
dados = pd.concat([pd.read_csv(l) for l in lista_csvs], ignore_index=True)


# Tratando os dados:
Em alguns campos não existe um padrão então vamos tratar:
* Deixar os Campos String todos com letra maiuscula
* Alterar o formato do valor para 2 casas depois da virgula
* Alterar o ano, pois está com um digito depois da virgula a idéia é nenhum.
* Ordenação dos dados por Ano.

In [64]:
## Tratando os datatypes:
#dados = dados.fillna(0)
dados = dados.astype({'cpf_cnpj': 'int','incentivador':'str','nro_projeto':'str','nome_projeto':'str','uf_projeto':'str','ano': 'int'})

## Deixando as string maiusculas para padronizar a busca
dados['incentivador'] = dados['incentivador'].str.upper()
dados['nome_projeto'] = dados['nome_projeto'].str.upper()
display(dados)

,cpf_cnpj,incentivador,nro_projeto,nome_projeto,uf_projeto,valor_incentivo,ano
156937,0,183598,CIRCO DOS SONHOS - TURNÊ 1,SP,40000.0,2019.0,0
156935,0,184336,Atos e Palhaços ArteClownizAção,SP,8000.0,2019.0,0
156934,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156933,0,211747,É NÓIS NA FITA - CURSO GRATUITO DE CINEMA 2022,SP,36000.0,2021.0,0
156932,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...
2909,11137051000186,INTERBELLE COMERCIO DE PRODUTOS DE BELEZA LTDA,212574,EMPREENDE AÍ: FIGURINO E MAQUIAGEM,SP,5000.0,2023
2910,12986300000170,S. R. GRANDO & CIA LTDA,212029,AURORA DOS TALENTOS,PR,403.3,2023
2911,10973413000106,BAZZE INDUSTRIA DE PERFIS EM PVC LTDA,211762,NATAL DA GENTE,RS,20000.0,2023
2904,14014761000107,"ROYAL BLUE COMERCIO, IMPORTACAO E EXPORTACAO LTDA",203833,ESPAÇO K 18 ANOS - UMA VIAGEM NO TEMPO,SP,2000.0,2023


# Verificando os registros cagados:
De todos os dados, foram encontrados 12 registros cagados onde o ano é igual a 0, o que indica uma quebra em algumas linnhas de alguns CSVs, mas foi coisa pouca, vou idenficar esses dados, extrair para um Excel, tratar manualmente esse 12 registros e trazer devolta para o dataframe final.

In [65]:
## Validando o numero de linhas por ano:
volume_linha_ano = dados.groupby(['ano'])['ano'].count()
display(volume_linha_ano)

## Gerando as linhas cagadas com o ano igual a zero:
dados_cagados = dados.loc[dados['ano'] == 0]
display(dados_cagados)

## Gerando um csv de nome linhas_cagadas.csv
arquivo_cagado='arquivo_cagado.csv'
dados_cagados.to_csv (arquivo_cagado, index = None, header=True)

ano
0         12
1993       2
1994      20
1995      84
1996    1018
1997    1774
1998    1881
1999    2042
2000    2398
2001    2346
2002    2652
2003    2891
2004    4047
2005    4778
2006    5826
2007    6524
2008    6360
2009    5983
2010    7436
2011    8081
2012    7856
2013    8075
2014    8045
2015    7411
2016    7091
2017    7580
2018    8402
2019    9185
2020    8516
2021    9293
2022    9315
2023      14
Name: ano, dtype: int64

,cpf_cnpj,incentivador,nro_projeto,nome_projeto,uf_projeto,valor_incentivo,ano
156937,0,183598,CIRCO DOS SONHOS - TURNÊ 1,SP,40000.0,2019.0,0
156935,0,184336,Atos e Palhaços ArteClownizAção,SP,8000.0,2019.0,0
156934,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156933,0,211747,É NÓIS NA FITA - CURSO GRATUITO DE CINEMA 2022,SP,36000.0,2021.0,0
156932,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156931,0,190965,O CASO MARTIN PICHE,RJ,36000.0,2021.0,0
156930,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156929,0,191847,É NÓIS NA FITA - CURSO GRATUITO DE CINEMA 2020,SP,18000.0,2020.0,0
156928,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156927,0,221192,Superações (Ano 2),SP,48000.0,2022.0,0


# Montando o dataframe com o CSV corrigido
Após corrigido, vamos montar um dataframe de nome `corrigido` e remover as linhas do consolidado que estão com o ano `0`

In [55]:
## Formatando os dados corrigidos
corrigido = pd.DataFrame()
corrigido = pd.read_csv('arquivo_cagado.csv')
corrigido['incentivador'] = corrigido['incentivador'].str.upper()
corrigido['nome_projeto'] = corrigido['nome_projeto'].str.upper()

## Removendo as linhas com ano 0 
dados = dados.loc[dados['ano'] != 0]

## Fazendo a junção e ordenando os dados por ano
dados = pd.concat([dados, corrigido], ignore_index=True)
dados = dados.sort_values('ano')

## Validação final pra ver se não ficou nada errado:
dados_cagados = dados.loc[dados['ano'] == 0]
display(dados_cagados)

,cpf_cnpj,incentivador,nro_projeto,nome_projeto,uf_projeto,valor_incentivo,ano
156937,0,183598,CIRCO DOS SONHOS - TURNÊ 1,SP,40000.0,2019.0,0
156935,0,184336,Atos e Palhaços ArteClownizAção,SP,8000.0,2019.0,0
156934,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156933,0,211747,É NÓIS NA FITA - CURSO GRATUITO DE CINEMA 2022,SP,36000.0,2021.0,0
156932,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156931,0,190965,O CASO MARTIN PICHE,RJ,36000.0,2021.0,0
156930,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156929,0,191847,É NÓIS NA FITA - CURSO GRATUITO DE CINEMA 2020,SP,18000.0,2020.0,0
156928,2043201000100,COZIMAX MÓVEIS MIRASSOL LTDA,0,0,0.0,0.0,0
156927,0,221192,Superações (Ano 2),SP,48000.0,2022.0,0


# Gerando o CSV final:
Com todos os campos tratados vamos gerar um CSV final consolidado com todos os dados.

In [56]:
arquivo_final='dados_consolidados.csv'
dados.to_csv (arquivo_final, index = None, header=True)

# Consultando registros por ano:

In [57]:
## Consultando Registros:
#a = int(input('Qual ano você deseja consultar?'))
#consulta = dados.loc[dados['ano'] == a]
#display(consulta)

# Consultando Registros por CNPJ

In [58]:
## Consultando Registros:
#a = int(input('Qual CNPJ você deseja consultar?'))
#consulta = dados.loc[dados['cpf_cnpj'] == a]
#display(consulta)

# Criando um arquivo Parquet do dataframe final
Vou criar um parquet ao invés de um CSV devido a velocidade de consulta e as possibilidades que ele vai trazer, a ídeia é posteriormente usar o formato Delta Lake.

In [59]:
## Gravando o Dataframe em Parquet
#caminho_parquet='DadosSalicNetParquet/'
#dados = dados.astype({'nro_projeto': 'int'})
#dados.to_parquet(caminho_parquet + 'dados.parquet.gzip',compression='gzip') 

# Total de incentivos por estado ao longo do tempo:

In [60]:
#dados.groupby('uf_projeto').valor_incentivo.sum().astype(int).sort_values(ascending=False)

In [61]:
import pandas as pd
import xlrd, os, glob, re, csv, shutil, codecs
from io import StringIO 

## Variaveis
path_airflow = '/Users/Anselmo/Documents/projeto_rescue/projetorescue/ingestao/airflow/dags/'
PastaXLS = path_airflow + 'dados/DadosSalicNetXLS'
PastaCSV = path_airflow + 'dados/DadosSalicNetCSV'
#lista_arquivos = os.listdir(PastaXLS)
arquivo_xls = PastaXLS + '/dados_pj_2022.xls'
ano = (re.findall("dados_pj_(\d+).xls", arquivo_xls))[0]
wb = xlrd.open_workbook(arquivo_xls, encoding_override='ISO-8859-1')
arquivoExcel = pd.read_excel(wb).assign(ano_corrente=ano)
nome = arquivo_xls.rpartition('.')[0]
#arquivo = nome + '.csv'


## Carregando o CSV pra memória
columns=['cpf_cnpj','incentivador','nro_projeto','nome_projeto','uf_projeto','valor_incentivo','ano']
var1 = None
arquivoExcel.to_csv (var1, index = None, header=False, sep=",")

## Da memoria pro Dataframe:
df = pd.DataFrame([row.split(',') for row in var1.split('\n')], columns=columns)
display(df)


AttributeError: 'NoneType' object has no attribute 'split'